# PSO

In [ ]:
import numpy as np 
import pandas as pd
import random
import time

In [ ]:
class PSO:
    def __init__(self, function, minimum = True, n_particles=10, iteration=100, n_dim=1, options={'c1':0.3, 'c2':0.5, 'w':0.9}, bound=[], lr_reduce=False):
        self.function = function
        self.minimum = minimum
        self.n_particles = n_particles
        self.iteration = iteration
        self.n_dim = n_dim
        self.options = options
        self.bound = bound
        self.lr_reduce = lr_reduce

    def optimize(self):
        u_teta = 0.9
        l_teta = 0.4
        #Inisialisasi awal posisi dan kecepatan partikel
        _position = np.zeros((self.n_particles, self.n_dim))
        _velocity = np.zeros((self.n_particles, self.n_dim))
        #print(self.bound)
        if self.bound is not None :
            for i in range(self.n_dim):
                _min  = self.bound[i][0]
                _max = self.bound[i][1]
                
                for j in range(self.n_particles):
                    _position[j][i] = random.uniform(_min, _max)
                    _velocity[j][i] = random.uniform(-abs(_max-_min), abs(_max-_min))
        else :
            for i in range(self.n_dim) :
                for j in range(self.n_particles):
                    _position[j][i] = random.uniform(0, 10)
                    _velocity[j][i] = random.uniform(-abs(10-0), abs(10-0))
        #print("Partikel Awal :"+str(_position))
        #print("Kecepatan Awal :"+str(_velocity))
        #Evaluasi nilai partikel terhadap fungsi
        #Tentukan Local Best dan Global Best
        #1. Evaluasi setiap posisi partikel terhadap fungsi
        #2. Tentukan Local Best dan Global Best
        #3. Local Best pada iterasi pertama adalah nilai posisi dari partikel itu sendiri
        #4. Global Best merupakan nilai dengan nilai fitness terbaik dari semua partikel
        #Local Best
        local_value = []
        local_best = []
        for i in _position:
            value = self.function(i)
            local_value.append(value)
            local_best.append(i)
        #Global Best
        best_value = max(local_value)
        global_best = local_best[local_value.index(max(local_value))]
        #Update kecepatan dan posisi partikel
        it = 0
        stop = False
        print("Update Partikel!!!")
        while(it < self.iteration):
            if stop :
                break
            else:
                #Pereduksian nilai teta (w)
                if self.lr_reduce:
                    self.options['w'] = u_teta - ((u_teta-l_teta)/self.iteration)*(it+1)
                for i in range(len(_position)):
                    #Membuat new_pos sesuai dengan n_dim, diisi dengan nilai 0
                    new_position = np.zeros((1, self.n_dim))[0]
                    for j in range(len(_position[i])):
                        r1 = random.uniform(0,1)
                        r2 = random.uniform(0,1)
                        new_vel = (self.options['w']*_velocity[i][j]) + (self.options['c1']*r1*(local_best[i]-_position[i][j]))+(self.options['c2']*r2*(global_best - _position[i][j]))
                        new_pos = new_vel + _position[i][j]
                        new_position[j] = new_pos[0]    
                        _velocity[i][j] = new_vel[0]
                        #new_position[j] = new_pos[0]
                    #Update Local Best dan Global Best dengan membandingkan nilai fitness posisi baru dengan localbest sebelumnya    
                    #Update Local Best
                    if self.minimum :
                        if self.function(new_position) < self.function(local_best[i]) :
                            local_best[i] = new_position
                            local_value[i] = self.function(new_position)
                    else :
                        if self.function(new_position) > self.function(local_best[i]) :
                            local_best[i] = new_position
                            local_value[i] = self.function(new_position)
                    #Update Global Best
                    if self.minimum :
                        best_value = min(local_value)
                        global_best = local_best[local_value.index(min(local_value))]
                    else :
                        best_value = max(local_value)
                        global_best = local_best[local_value.index(max(local_value))]
            it += 1
        #Return value
        return global_best, best_value

# Input Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df= pd.read_excel('/content/drive/My Drive/dataset/Skripsi/data_v3.xlsx') 
df.head()

,Tweet,Anotator 1 (Trifebi Shina),Anotator 2 (Mujaddid),Anotator 3 (Harris Ishaq),Sentimen Final
0,Alat penjajah sekarang adalah UU cipta kerja >...,Negatif,0,Negatif,Negatif
1,Aroma kepentingan asing dan pemilik modal sang...,Negatif,0,Negatif,Negatif
2,"Serentak di seluruh Nusantara, buruh menolak U...",Negatif,0,Negatif,Negatif
3,"Menjajah bangsa sendiri, usulan pemerintah zal...",Negatif,0,Negatif,Negatif
4,Negeri ini belum merdeka ..uu perbudakan lahir...,Negatif,0,Negatif,Negatif


In [ ]:
df.drop(['Anotator 1 (Trifebi Shina)', 'Anotator 2 (Mujaddid)', 'Anotator 3 (Harris Ishaq)'], axis=1, inplace=True)
df.rename(columns = {'Sentimen Final':'Sentimen'}, inplace = True)
df.head()

,Tweet,Sentimen
0,Alat penjajah sekarang adalah UU cipta kerja >...,Negatif
1,Aroma kepentingan asing dan pemilik modal sang...,Negatif
2,"Serentak di seluruh Nusantara, buruh menolak U...",Negatif
3,"Menjajah bangsa sendiri, usulan pemerintah zal...",Negatif
4,Negeri ini belum merdeka ..uu perbudakan lahir...,Negatif


In [ ]:
df.isnull().sum()

Tweet       0
Sentimen    0
dtype: int64

In [ ]:
df.shape

(1000, 2)

# Preprocessing

In [ ]:
pip install Sastrawi

     |████████████████████████████████| 215kB 8.3MB/s 


In [ ]:
#Import library untuk proses preprocessing
pd.set_option('display.max_colwidth', None)
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
#Case folding
df['Tweet'] = df['Tweet'].str.lower()
df.head(5)

,Tweet,Sentimen
0,alat penjajah sekarang adalah uu cipta kerja >> #buruhmenggugatuuciptakerja << #tolakuuciptakerja >> pic.twitter.com/wpdhlo3djq,Negatif
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja. tolak uu cipta kerja >> #buruhmenggugatuuciptakerja << #uuciptakerjaalatpenjajahan#tolakuuciptakerja >> pic.twitter.com/i0p15fg9ba,Negatif
2,"serentak di seluruh nusantara, buruh menolak uu yang baru di sahkan yang meresahkan pekerja >> #buruhmenggugatuuciptakerja << #tolakuuciptakerja >> pic.twitter.com/whlraa20ck",Negatif
3,"menjajah bangsa sendiri, usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat >> #buruhmenggugatuuciptakerja << #tolakuuciptakerja >> pic.twitter.com/zqlu4ej483",Negatif
4,negeri ini belum merdeka ..uu perbudakan lahir kembali oleh pemerintah zalim >> #buruhmenggugatuuciptakerja << #tolakuuciptakerja >> pic.twitter.com/zgcixpzqdh,Negatif


In [ ]:
#Cleansing
def cleansing(docs):
  tweet = re.sub(r'(http\S+|https\S+|twitter.com\S+|pic.twitter\S+)', '', docs)
  clean_tweet = tweet.replace('\n', '')
  clean_tweet = re.sub('(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)|([^A-Za-z \t])|(\w+:\/\/\S+)','', clean_tweet)
  return clean_tweet

In [ ]:
df['Tweet'] = df['Tweet'].apply(cleansing)
df.head()

,Tweet,Sentimen
0,alat penjajah sekarang adalah uu cipta kerja,Negatif
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja tolak uu cipta kerja,Negatif
2,serentak di seluruh nusantara buruh menolak uu yang baru di sahkan yang meresahkan pekerja,Negatif
3,menjajah bangsa sendiri usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat,Negatif
4,negeri ini belum merdeka uu perbudakan lahir kembali oleh pemerintah zalim,Negatif


In [ ]:
#Tokenization
def tokenize(docs):
    return word_tokenize(docs)

In [ ]:
df['Token Tweet'] = df['Tweet'].apply(tokenize)
df.head()

,Tweet,Sentimen,Token Tweet
0,alat penjajah sekarang adalah uu cipta kerja,Negatif,"[alat, penjajah, sekarang, adalah, uu, cipta, kerja]"
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja tolak uu cipta kerja,Negatif,"[aroma, kepentingan, asing, dan, pemilik, modal, sangat, terasa, dalam, uu, cipta, kerja, tolak, uu, cipta, kerja]"
2,serentak di seluruh nusantara buruh menolak uu yang baru di sahkan yang meresahkan pekerja,Negatif,"[serentak, di, seluruh, nusantara, buruh, menolak, uu, yang, baru, di, sahkan, yang, meresahkan, pekerja]"
3,menjajah bangsa sendiri usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat,Negatif,"[menjajah, bangsa, sendiri, usulan, pemerintah, zalim, dan, disetujui, dewan, pengkhianat, rakyat]"
4,negeri ini belum merdeka uu perbudakan lahir kembali oleh pemerintah zalim,Negatif,"[negeri, ini, belum, merdeka, uu, perbudakan, lahir, kembali, oleh, pemerintah, zalim]"


In [ ]:
#Stopword removal
def stop_removal(docs):
  words = stopwords.words('indonesian')
  words.extend(['amp', 'sih', 'nih', 'nya', 'n', 'nn', 't', 'hehe', 'nan', 'loh', '&amp']) #Nanti ditambahin juga yang remove itu, karena ada beberapa kata penting dlm list stopword
  words.remove('tidak')
  words = set(words)
  return [word for word in docs if word not in words]

In [ ]:
df['Token Tweet'] = df['Token Tweet'].apply(stop_removal)
df.head()

,Tweet,Sentimen,Token Tweet
0,alat penjajah sekarang adalah uu cipta kerja,Negatif,"[alat, penjajah, uu, cipta, kerja]"
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja tolak uu cipta kerja,Negatif,"[aroma, kepentingan, asing, pemilik, modal, uu, cipta, kerja, tolak, uu, cipta, kerja]"
2,serentak di seluruh nusantara buruh menolak uu yang baru di sahkan yang meresahkan pekerja,Negatif,"[serentak, nusantara, buruh, menolak, uu, sahkan, meresahkan, pekerja]"
3,menjajah bangsa sendiri usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat,Negatif,"[menjajah, bangsa, usulan, pemerintah, zalim, disetujui, dewan, pengkhianat, rakyat]"
4,negeri ini belum merdeka uu perbudakan lahir kembali oleh pemerintah zalim,Negatif,"[negeri, merdeka, uu, perbudakan, lahir, pemerintah, zalim]"


In [ ]:
#Stemming
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stemming(docs):
  return [stemmer.stem(word) for word in docs]

In [ ]:
df['Token Tweet'] = df['Token Tweet'].apply(stemming)
df.head()

,Tweet,Sentimen,Token Tweet
0,alat penjajah sekarang adalah uu cipta kerja,Negatif,"[alat, jajah, uu, cipta, kerja]"
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja tolak uu cipta kerja,Negatif,"[aroma, penting, asing, milik, modal, uu, cipta, kerja, tolak, uu, cipta, kerja]"
2,serentak di seluruh nusantara buruh menolak uu yang baru di sahkan yang meresahkan pekerja,Negatif,"[serentak, nusantara, buruh, tolak, uu, sahkan, resah, kerja]"
3,menjajah bangsa sendiri usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat,Negatif,"[jajah, bangsa, usul, perintah, zalim, tuju, dewan, khianat, rakyat]"
4,negeri ini belum merdeka uu perbudakan lahir kembali oleh pemerintah zalim,Negatif,"[negeri, merdeka, uu, budak, lahir, perintah, zalim]"


# TF-IDF

In [ ]:
def join_text_list(texts):
    return ' '.join([text for text in texts])
df["Final Tweet"] = df["Token Tweet"].apply(join_text_list)
df.head()

,Tweet,Sentimen,Token Tweet,Final Tweet
0,alat penjajah sekarang adalah uu cipta kerja,Negatif,"[alat, jajah, uu, cipta, kerja]",alat jajah uu cipta kerja
1,aroma kepentingan asing dan pemilik modal sangat terasa dalam uu cipta kerja tolak uu cipta kerja,Negatif,"[aroma, penting, asing, milik, modal, uu, cipta, kerja, tolak, uu, cipta, kerja]",aroma penting asing milik modal uu cipta kerja tolak uu cipta kerja
2,serentak di seluruh nusantara buruh menolak uu yang baru di sahkan yang meresahkan pekerja,Negatif,"[serentak, nusantara, buruh, tolak, uu, sahkan, resah, kerja]",serentak nusantara buruh tolak uu sahkan resah kerja
3,menjajah bangsa sendiri usulan pemerintah zalim dan disetujui dewan pengkhianat rakyat,Negatif,"[jajah, bangsa, usul, perintah, zalim, tuju, dewan, khianat, rakyat]",jajah bangsa usul perintah zalim tuju dewan khianat rakyat
4,negeri ini belum merdeka uu perbudakan lahir kembali oleh pemerintah zalim,Negatif,"[negeri, merdeka, uu, budak, lahir, perintah, zalim]",negeri merdeka uu budak lahir perintah zalim


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 

tf_idf = TfidfVectorizer(binary=True)
tfidf_mat = tf_idf.fit_transform(df["Final Tweet"]).toarray()

In [ ]:
tfidf = pd.DataFrame(tfidf_mat, columns=tf_idf.get_feature_names())
tfidf.head()

,aamiin,abai,abdi,abie,abk,absurd,acu,ada,adab,adart,adat,adek,adidaya,adil,administrasi,adu,agal,agama,agamawan,agenda,agraria,agung,ah,ahli,air,airlangga,aja,ajak,ajar,aju,akademik,akademisi,akal,akan,akhir,akhirat,akhlak,akibat,akomodasi,akselerasi,...,wakil,waktu,walkie,wapres,waras,warga,waris,warna,waspada,well,wenang,widodo,wikipedia,wilayah,wkwk,wna,woi,wow,woy,wuhan,wujud,xcxajc,xfxfxaxa,xfxfxxaexfxfxxan,ya,yah,yakali,yakin,yangg,yes,yg,yng,yra,yth,yuana,yudicial,yuk,zalim,zero,zqn
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.292791,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.347168,0.0,0.0


In [ ]:
X = tfidf.values
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
label = np.where(df['Sentimen']=='Positif', 1, 0)
y = label
y[45:75]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

# Klasifikasi SVM-PSO (Iterasi 50x) - Skenario 3

In [ ]:
from sklearn import model_selection, svm
from sklearn.model_selection import KFold

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

from scipy.sparse import csr_matrix as matrixTransform

In [ ]:
accDum = []
presDum = []
recDum = []
cmDum = []
modelDum = []
def classify(x):   
    SVM = svm.SVC(gamma=abs(x[0]), C=abs(x[1]), kernel='rbf', probability=True)
    SVM.fit(Train_X_final, y_train)
    predictions_SVM = SVM.predict(Test_X_final)
    CM = confusion_matrix(y_test, predictions_SVM)
    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    try :
        pres = TP/(TP+FP)
    except :
        print("")
    accu = (TP+TN)/(TP+FP+TN+FN)
    recall = TP/(TP+FN)
    accDum.append(accu)
    recDum.append(recall)
    presDum.append(pres)
    cmDum.append(CM)
    modelDum.append(SVM)

    #obj.setRecall(recall)
    #obj.setPres(pres)
    
    inv_acc = 1/accu
    return inv_acc

In [ ]:
def optimizing():
    option = {'c1':0.3, 'c2':0.5, 'w':0.9}
    bound = [(0, 1),(0.1, 10)]
    optimizer = PSO(classify, n_particles=10, n_dim=2, options=option, iteration=50, bound=bound, lr_reduce=True)
    param, result = optimizer.optimize()
    acc = 1/result
    print("Best Param :", param)
    return acc

In [ ]:
acc_scores = []
presisi_scores = []
recall_scores = []
models = []
cv = KFold(n_splits=10, random_state=32, shuffle=True)
for train_index, test_index in cv.split(X):
    X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
    #Encoder = LabelEncoder()
    #y_train = Encoder.fit_transform(y_train)
    #y_test = Encoder.fit_transform(y_test)

    Train_X_final = matrixTransform(X_train)
    Test_X_final = matrixTransform(X_test)   
    print("Data Train :", len(X_train))
    print("Data Test :", len(X_test))

    accDum = []
    presDum = []
    recDum = []
    cmDum = []
    modelDum = []
    optimizing()

    acc_scores.append(max(accDum))
    presisi_scores.append(presDum[accDum.index(max(accDum))])
    recall_scores.append(recDum[accDum.index(max(accDum))])
    confusion = cmDum[accDum.index(max(accDum))]
    models.append(modelDum[accDum.index(max(accDum))])
    print("Confusion Matrix")
    print(confusion)

Data Train : 900
Data Test : 100


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in long_scalars
  app.launch_new_instance()


Update Partikel!!!
Best Param : [0.17696122 9.03250209]
Confusion Matrix
[[43  5]
 [ 6 46]]
Data Train : 900
Data Test : 100
Update Partikel!!!
Best Param : [ 0.08503555 -5.99551964]
Confusion Matrix
[[43  2]
 [ 0 55]]
Data Train : 900
Data Test : 100
Update Partikel!!!
Best Param : [ 0.52964696 -7.25253165]
Confusion Matrix
[[44  1]
 [ 7 48]]
Data Train : 900
Data Test : 100
Update Partikel!!!
Best Param : [0.41399545 6.86303492]
Confusion Matrix
[[55  0]
 [ 2 43]]
Data Train : 900
Data Test : 100
Update Partikel!!!
Best Param : [1.3410166  1.91817363]
Confusion Matrix
[[47  0]
 [ 2 51]]
Data Train : 900
Data Test : 100
Update Partikel!!!
Best Param : [-0.0618481  -6.98542766]
Confusion Matrix
[[44  0]
 [ 2 54]]
Data Train : 900
Data Test : 100
Update Partikel!!!
Best Param : [ 0.29335517 -1.74615475]
Confusion Matrix
[[56  3]
 [ 2 39]]
Data Train : 900
Data Test : 100
Update Partikel!!!
Best Param : [ 0.23057    -6.32239989]
Confusion Matrix
[[53  1]
 [ 3 43]]
Data Train : 900
Data T

In [ ]:
print("Pengujian dengan optimasi...")
print("Accuracy :", acc_scores)
print("Presisi :", presisi_scores)
print("Recall :", recall_scores)
print("Accuracy Average :", np.mean(acc_scores))
print("Presisi Average :", np.mean(presisi_scores))
print("Recall Average :", np.mean(recall_scores))

Pengujian dengan optimasi...
Accuracy : [0.89, 0.98, 0.92, 0.98, 0.98, 0.98, 0.95, 0.96, 0.9, 0.96]
Presisi : [0.9019607843137255, 0.9649122807017544, 0.9795918367346939, 1.0, 1.0, 1.0, 0.9285714285714286, 0.9772727272727273, 0.9743589743589743, 0.9791666666666666]
Recall : [0.8846153846153846, 1.0, 0.8727272727272727, 0.9555555555555556, 0.9622641509433962, 0.9642857142857143, 0.9512195121951219, 0.9347826086956522, 0.8085106382978723, 0.94]
Accuracy Average : 0.95
Presisi Average : 0.9705834698619971
Recall Average : 0.9273960837315969


In [ ]:
modelSVM = models[acc_scores.index(max(acc_scores))]
print("Model SVM: ")
print(modelSVM)

Model SVM: 
SVC(C=5.995519644750603, break_ties=False, cache_size=200, class_weight=None,
    coef0=0.0, decision_function_shape='ovr', degree=3,
    gamma=0.08503555172486893, kernel='rbf', max_iter=-1, probability=True,
    random_state=None, shrinking=True, tol=0.001, verbose=False)
